In [1]:
df_sun = pd.read_csv('E:/Learning_Spoons_Second_Project/data/일조량.csv')
df_tomato = pd.read_csv('E:/Learning_Spoons_Second_Project/data/토마토_도매가격_fin.csv')
df_time = pd.read_csv('E:/Learning_Spoons_Second_Project/data/날씨_시계열.csv', encoding='euc-kr')

#수정
df_tomato_copy = df_tomato.drop(['연도','파종 연도'],axis=1).copy()

df_tomato_copy

#수정(merge를 위해 컬럼명 변경)
df_tomato_copy = df_tomato_copy.rename(columns={'일시':'판매일시','파종 예상 일시':'일시'})

# 머신러닝을 위해 도를 int 형식으로 변환을 위한 dictionary
dict_1 = {'강원도':0, '경기도':1, '인천광역시':2, '서울특별시':3, '경상북도':4, '충청북도':5, '충청남도':6, '대전광역시':7,
       '전라북도':8, '대구광역시':9, '울산광역시':10, '경상남도':11, '광주광역시':12, '부산광역시':13, '전라남도':14,
       '제주특별자치도':15}

# 도_num 컬럼을 dict_1을 이용하여 생성
# 수정
df_tomato_copy ['도_num'] = df_tomato_copy['도'].map(dict_1)
df_sun['도_num'] = df_sun['도'].map(dict_1)
df_time['도_num'] = df_time['도'].map(dict_1)

NameError: name 'pd' is not defined

In [ ]:

df_tomatoma = pd.merge(df_air, df_tomato_copy, on=['일시','도_num'], how='left')

In [ ]:
# 컬렁명을 명확하게 구분하기 위해 변경
df_tomatoma.rename(columns={'일시':'파종일시'},inplace=True)

In [ ]:
#Null 값을 채우기 위해 datetime 형식으로 변경
df_tomatoma['파종일시'] = pd.to_datetime(df_tomatoma['파종일시'])
df_tomatoma['판매일시'] = pd.to_datetime(df_tomatoma['판매일시'])

In [ ]:
#판매일시가 null 값인 것들을 파종일시에서 4개월 후의 일자로 채워넣음
df_tomatoma['판매일시'] = df_tomatoma['판매일시'].fillna(df_tomatoma['파종일시']+pd.DateOffset(120))

In [ ]:
# 판매일시를 index로 변경
#죄송합니다 제가 df_tomatoma_final을 안쓰고 그냥 df_tomatoma로 다 바꿨어요
df_tomatoma = df_tomatoma.set_index('판매일시')

In [ ]:
# 변경이 되었는지 확인
df_tomatoma

In [ ]:
# 각 권역의 마켓명이 같을 것이라고 가정하여 fillna의 bfill을 사용하여 마켓명 채우기위한 함수
# 그냥 복사해서 붙여넣으시면 됩니다
def fill_market(lst,frame):
    frame.loc[frame['도'].isin(lst),'마켓명'] =  frame.loc[frame['도'].isin(lst),'마켓명'].fillna(method='bfill')
    return frame.head(10)

In [ ]:
# 함수를 활용하여 마켓명 nan값 처리
#이것도 복붙
fill_market(광주_lst,df_tomatoma)
fill_market(대구_lst,df_tomatoma)
fill_market(부산_lst,df_tomatoma)
fill_market(대전_lst,df_tomatoma)
fill_market(서울_lst,df_tomatoma)

In [ ]:
# 함수를 사용하였지만 값이 안들어간 마켓명이 존재하여 하드코딩으로 값 채움
# 아마 이게 df_tomatoma_final로 되어있을거에요
df_tomatoma.loc[df_tomatoma['마켓명'].isnull(),'마켓명'] = '각화도매'

In [ ]:
fill_price(df_tomatoma)

In [ ]:
# 저장
df_tomatoma.to_csv(('E:/Learning_Spoons_Second_Project/data/final_data.csv',encoding='cp949')

In [ ]:
# 저장한 데이터 확인
pd.read_csv('E:/Learning_Spoons_Second_Project/data/final_data.csv',encoding='cp949')